In [ ]:
# installing required libraries
!pip install geopandas
!add-apt-repository ppa:ubuntugis/ppa -y
!apt-get update
!apt-get install python-numpy gdal-bin libgdal-dev python3-rtree
!pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import skimage
import matplotlib.pyplot as plt
import numpy as np


%cd /content/gdrive/MyDrive/SpaceNet
import libs.solaris as sol # modified solaris library
from utils.dist import *
from data.dataset import *
from models.unet import *
from glob import glob

In [ ]:
# load data and define transform
import torchvision
import torchvision.transforms as transforms

dataset = Spacenet()

train, test = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)])
train_loader = torch.utils.data.DataLoader(train, batch_size = 12, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size= 12, shuffle=True, num_workers=2)

In [ ]:
# baseline model
device = 'cuda'

net = Unet(8, 2)
net = net.to(device)
#cudnn.benchmark = True

file_name = 'Unet_khartoum.pt'
trained_dir = '/content/gdrive/MyDrive/SpaceNet/trained'

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay = 0.1)

In [ ]:
# train and evaluate
from utils.eval import *
import math
val_loss = []

def train(epoch):
  print('\n Current train epoch : ', epoch)
  net.train()
  for batch_idx, (imgs, targets) in enumerate(train_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
  
    loss.backward()
    if math.isnan(loss.item()):
      raise KeyboardInterrupt('nan detected')
    print('loss: ', loss.item())
    optimizer.step()

def test(epoch):
  print('\n current validation epoch: ', epoch)
  net.eval()
  total_loss = 0
  for batch_idx, (imgs, targets) in enumerate(validation_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
    total_loss += loss.item()/10 # divide by batch size 

  state = {
      'net': net.state_dict(),
      'epoch': epoch,
      'optimizer': optimizer.state_dict()
  }
  torch.save(state, os.path.join(trained_dir, file_name))
  val_loss.append(total_loss)
  print('\n Current loss : ', total_loss) # get segmentation metrics 

In [ ]:
for epoch in range(1, 60):
  train(epoch)
  test(epoch)